==============================================================================
Mini Projet - Analyse de données du Burkina Faso
==============================================================================
  Ce script Python réalise toutes les étapes de l'exercice :
 1. Prétraitement et filtrage d'un fichier Geonames.org pour le Burkina Faso.
 2. Analyse et extraction de sous-ensembles de données.
 3. Consolidation des résultats dans un fichier Excel.

Étape 1 : Configuration, vérification et importation des bibliothèques
Ce bloc de code assure que toutes les bibliothèques requises sont installées.
L'importation de 'os' est faite en premier pour pouvoir utiliser 'os.system'.

In [26]:
import os
import sys

# La liste des bibliothèques requises
required_libraries = ['pandas', 'requests', 'openpyxl']
installed_libraries = []

print("Vérification des bibliothèques requises...")

# Vérifier si chaque bibliothèque est installée
for lib in required_libraries:
    try:
        __import__(lib)
        installed_libraries.append(lib)
    except ImportError:
        print(f"La bibliothèque '{lib}' n'est pas installée. Installation en cours...")
        os.system(f'{sys.executable} -m pip install {lib}')
        try:
            __import__(lib)
            print(f"'{lib}' a été installée avec succès.")
        except ImportError:
            print(f"Erreur: Impossible d'installer '{lib}'. Veuillez l'installer manuellement avec 'pip install {lib}'.")
            # Arrêter le script si l'installation échoue
            exit()

# Re-importer toutes les bibliothèques une fois que tout est prêt
import pandas as pd
import requests
import zipfile
import io
import openpyxl

print("Toutes les bibliothèques requises sont installées et prêtes à être utilisées.")
print("-" * 50)

Vérification des bibliothèques requises...
Toutes les bibliothèques requises sont installées et prêtes à être utilisées.
--------------------------------------------------


===========================================================================
Étape 2 : Téléchargement, décompression et prétraitement des données brutes
===========================================================================

In [27]:
url = 'http://download.geonames.org/export/dump/BF.zip'
print("Téléchargement et décompression du fichier BF.zip...")

# Télécharger le fichier et l'extraire
try:
    r = requests.get(url, stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    print("Fichier BF.zip décompressé avec succès.")
except requests.exceptions.RequestException as e:
    print(f"Erreur de téléchargement : {e}")
    # Arrêter le script si le téléchargement échoue
    exit()

# Le fichier est délimité par des tabulations et n'a pas d'en-tête.
# Le fichier BF.txt est situé dans le sous-dossier 'BF' après l'extraction.
file_path = os.path.join('BF.txt')
print(f"Lecture du fichier {file_path}...")

# Lire le fichier BF.txt en tant que DataFrame
df_raw = pd.read_csv(file_path, sep='\t', header=None, encoding='utf-8')

# Le README de Geonames indique que les colonnes qui nous intéressent sont :
# - geonameid (colonne 0)
# - name (colonne 1)
# - latitude (colonne 4)
# - longitude (colonne 5)
df_filtered = df_raw[[0, 1, 4, 5]].copy()

# Renommer les colonnes pour une meilleure lisibilité
df_filtered.columns = ['ID', 'location_name', 'lat', 'long']

# Sauvegarder le DataFrame filtré dans un nouveau fichier CSV
df_filtered.to_csv('burkina_location.csv', index=False, encoding='utf-8')

print("Fichier 'burkina_location.csv' créé avec succès.")
print("\nAperçu des 5 premières lignes du nouveau fichier :")
print(df_filtered.head())
print("-" * 50)

Téléchargement et décompression du fichier BF.zip...
Fichier BF.zip décompressé avec succès.
Lecture du fichier BF.txt...
Fichier 'burkina_location.csv' créé avec succès.

Aperçu des 5 premières lignes du nouveau fichier :
        ID       location_name       lat     long
0  2282318              Pouéné   9.72908 -2.78660
1  2285251  Léraba Occidentale  10.28333 -5.11667
2  2287216           Kéléouoro   9.80748 -4.05023
3  2294066         White Volta   8.70194 -0.99056
4  2298457        Issana Bouga  10.91667 -1.18333
--------------------------------------------------


===========================================================================
Étape 3 : Opérations sur le fichier CSV
===========================================================================

In [28]:

# Charger le fichier CSV pour les analyses suivantes
df_burkina = pd.read_csv('burkina_location.csv')

# 3.1 Extraire les données contenant le nom 'gounghin'
# La méthode str.contains() permet de chercher une sous-chaîne dans la colonne
df_gounghin = df_burkina[df_burkina['location_name'].str.contains('gounghin', case=False, na=False)]
df_gounghin.to_csv('gounghin.csv', index=False)
print("Fichier 'gounghin.csv' créé avec les données contenant le nom 'gounghin'.")
print("\nDonnées pour 'gounghin' :")
print(df_gounghin)
print("-" * 50)


# 3.2 Extraire les lieux dont les premières lettres sont comprises entre 'A' et 'P'
# Utiliser .str.upper() et .between() pour filtrer les noms de lieux
df_a_to_p = df_burkina[df_burkina['location_name'].str.upper().str[0].between('A', 'P')]
print("Aperçu des lieux dont le nom commence par une lettre de 'A' à 'P' :")
print(df_a_to_p.head())
print("-" * 50)

Fichier 'gounghin.csv' créé avec les données contenant le nom 'gounghin'.

Données pour 'gounghin' :
             ID                  location_name       lat     long
153     2353306                       Gounghin  12.06677 -1.42134
7269    2360473                       Gounghin  12.62488 -1.36398
10260   2570204                       Gounghin  12.31436 -1.37900
10746  10342749                       Gounghin  12.06667 -0.15000
10759  10629032             BICIAB // Gounghin  12.35921 -1.54273
10818  11257296            Gounghin Department  12.06671 -0.15484
10845  11900526                  Gounghin Nord  12.36120 -1.55055
10846  11900528  Zone Industrielle de Gounghin  12.36631 -1.54137
10852  11900619                   Gounghin Sud  12.35298 -1.54342
10866  11900680                       Gounghin  12.35895 -1.54442
--------------------------------------------------
Aperçu des lieux dont le nom commence par une lettre de 'A' à 'P' :
          ID          location_name       lat     long